# SD212: Graph mining
## Lab 3: Graph clustering

In this lab, you will learn to cluster a graph by Louvain and interpret the results. 

## Import

In [ ]:
from IPython.display import SVG

In [ ]:
import numpy as np
from scipy import sparse

In [ ]:
from sknetwork.data import load_netset, grid, karate_club
from sknetwork.clustering import Louvain, modularity
from sknetwork.ranking import PageRank, top_k
from sknetwork.linalg import normalize
from sknetwork.utils import membership_matrix
from sknetwork.visualization import svg_graph, svg_digraph, svg_bigraph

## Data

We will work on the following graphs (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Openflights (graph)
* WikiVitals (digraph)
* Cinema (bigraph)

In [ ]:
openflights = load_netset('openflights')
wikivitals = load_netset('wikivitals')
cinema = load_netset('cinema')

## 1. Graphs

The Louvain algorithm aims at maximizing [modularity](https://en.wikipedia.org/wiki/Modularity_(networks)).

## Grid

Consider a grid:

In [ ]:
graph = grid(9, 9, True)
adjacency = graph.adjacency
position = graph.position

In [ ]:
image = svg_graph(adjacency, position, width=200, height=200)
SVG(image)

## To do

* Cluster the graph by Louvain and return the corresponding modularity.
* Try to shuffle the nodes and observe the results.
* Find a better clustering than Louvain in terms of modularity.

In [ ]:
louvain = Louvain()

In [ ]:
labels = louvain.fit_transform(adjacency)

In [ ]:
image = svg_graph(adjacency, position, labels=labels, width=200, height=200)
SVG(image)

In [ ]:
modularity(adjacency, labels, return_all=True)

In [ ]:
adjacency

In [ ]:
labels = np.array([3 * (i // 3) + j // 3 for i in range(9) for j in range(9)])

In [ ]:
image = svg_graph(adjacency, position, labels=labels, width=200, height=200)
SVG(image)

In [ ]:
modularity(adjacency, labels, return_all=True)

## Karate Club


The [karate club graph](https://en.wikipedia.org/wiki/Zachary%27s_karate_club) provides ground-truth clusters.

In [ ]:
graph = karate_club(metadata=True)

In [ ]:
adjacency = graph.adjacency
position = graph.position
labels_true = graph.labels

In [ ]:
image = svg_graph(adjacency, position, labels=labels_true)
SVG(image)

## To do

* Cluster the graph by Louvain and display the labels.
* Give the modularity.
* Display the aggregate graph (use ``display_node_weight`` to show the volume of each cluster).
* Display the bipartite graph between clusters and ground-truth labels (use sparse matrix multiplications to get the biadjacency matrix).
* Adapt the resolution to get 2 clusters and repeat the experiments.

In [ ]:
louvain = Louvain(resolution=0.5)

In [ ]:
labels = louvain.fit_transform(adjacency)

In [ ]:
image = svg_graph(adjacency, position, labels=labels)
SVG(image)

In [ ]:
image = svg_graph(adjacency, position, labels=labels_true)
SVG(image)

In [ ]:
adjacency_aggregate = louvain.aggregate_

In [ ]:
adjacency_aggregate

In [ ]:
membership = membership_matrix(labels)

In [ ]:
membership.shape

In [ ]:
# averaging
averaging = normalize(membership.T.tocsr()).T

In [ ]:
averaging.shape

In [ ]:
# position of barycenters of each cluster
position_aggregate = averaging.T.dot(position)

In [ ]:
image = svg_graph(adjacency_aggregate, position_aggregate, labels=np.arange(len(set(labels))), display_node_weight=True)
SVG(image)

In [ ]:
membership.shape

In [ ]:
membership_true = membership_matrix(labels_true).astype(int)

In [ ]:
membership_true.shape

In [ ]:
biadjacency_cluster = membership_true.T.dot(membership).tocsr()

In [ ]:
biadjacency_cluster

In [ ]:
biadjacency_cluster.toarray()

In [ ]:
np.unique(labels, return_counts=True)

In [ ]:
image = svg_bigraph(biadjacency_cluster)

In [ ]:
SVG(image)

## Openflights


In [ ]:
graph = openflights

In [ ]:
adjacency = graph.adjacency
position = graph.position
names = graph.names

In [ ]:
image = svg_graph(adjacency, position, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

## To do

* Display the same world map with the clusters found by Louvain (resolution 1).
* How many clusters are there?
* What is the Simpson index of this clustering? Check the function ``modularity``.<br>
How do you interpret the *inverse* of the Simpson index?
* List the top airport of each cluster in number of flights.
* Display the aggregate graph with the cluster volumes and the name of the top airport of each cluster.
* Display the same graph restricted to clusters with at least 100 nodes.
* Which of these clusters has the highest inside / outside / total traffic in number of flights?
* Which of these clusters is the strongest?<br>
Display the original graph with the strength of each cluster.

In [ ]:
louvain = Louvain(resolution=1)

In [ ]:
labels = louvain.fit_transform(adjacency)

In [ ]:
image = svg_graph(adjacency, position, labels=labels, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

In [ ]:
len(set(labels))

In [ ]:
modularity(adjacency, labels, return_all=True)

In [ ]:
traffic = adjacency.dot(np.ones(len(names)))

In [ ]:
# daily number of flights
traffic

In [ ]:
names[top_k(traffic, 10)]

In [ ]:
# top airport per cluster
top_airports = []
for label in np.unique(labels):
    index = np.argwhere(labels==label).ravel()
    top_airports.append(index[np.argmax(traffic[index])])
top_airports = np.array(top_airports)

In [ ]:
names[top_airports]

In [ ]:
adjacency_aggregate = louvain.aggregate_

In [ ]:
membership = membership_matrix(labels)
averaging = normalize(membership.T.tocsr())

In [ ]:
averaging.shape

In [ ]:
position_aggregate = averaging.dot(position)

In [ ]:
_, counts = np.unique(labels, return_counts=True)

In [ ]:
index = np.argwhere(counts >= 100).ravel()

In [ ]:
image = svg_graph(adjacency_aggregate[index][:, index], position_aggregate[index], labels=index, 
                  names=names[top_airports][index], width=800, height=400, display_node_weight=True, 
                  display_edge_weight=True, edge_width_max=200)

In [ ]:
SVG(image)

In [ ]:
adjacency_aggregate

In [ ]:
# total weight inside each cluster 
total_weights_in = adjacency_aggregate.diagonal()

In [ ]:
# total weight of each cluster 
n_cluster = adjacency_aggregate.shape[0]
total_weights = adjacency_aggregate.dot(np.ones(n_cluster))

In [ ]:
# strengths
strengths = total_weights_in / total_weights

In [ ]:
# strongest cluster
names[top_airports[np.argmax(strengths)]]

In [ ]:
# weakest cluster
names[top_airports[np.argmin(strengths)]]

In [ ]:
# display strengths as scores
image = svg_graph(adjacency, position, scores=strengths[labels], 
                  width=800, height=400, node_size=3, display_edges=False)
SVG(image)

## 2. Directed graphs

## Wikipedia Vitals

In [ ]:
graph = wikivitals

In [ ]:
adjacency = graph.adjacency
names = graph.names

## To do

* Cluster the graph by Louvain (resolution 1).
* List the top-5 pages of each cluster in terms of Personalized PageRank.
* Display the aggregate graph with the cluster volumes and the name of the top page of each cluster.
* Display the same graph restricted to clusters with at least 100 nodes.
* Which of these clusters is the strongest? the weakest?
* Display the bipartite graph linking these clusters to their ground-truth labels.
* Display the same graph restricted to the main ground-truth labels of each cluster, each representing at least 10% of the labels of the cluster.
* Repeat the same experiments at resolution 2.

In [ ]:
louvain = Louvain(resolution=1)

In [ ]:
labels = louvain.fit_transform(adjacency)

In [ ]:
pagerank = PageRank()

In [ ]:
top_pages = []
for label in np.unique(labels):
    scores = pagerank.fit_transform(adjacency, seeds=labels==label)
    scores *= labels==label
    print(names[top_k(scores, 5)])
    top_pages.append(np.argmax(scores))

In [ ]:
top_pages = np.array(top_pages)

In [ ]:
aggregate = louvain.aggregate_

In [ ]:
_, counts = np.unique(labels, return_counts=True)
index = np.argwhere(counts >= 100).ravel()

In [ ]:
image = svg_graph(aggregate[index][:, index], names=names[top_pages][index], labels=index, 
                  display_edge_weight=True, display_node_weight=True, edge_width_max=100)

In [ ]:
SVG(image)

In [ ]:
labels_true = graph.labels
names_labels_true = graph.names_labels

In [ ]:
membership_true = membership_matrix(labels_true).astype(int)

In [ ]:
membership = membership_matrix(labels)

In [ ]:
biadjacency_cluster = membership_true.T.dot(membership).tocsr()

In [ ]:
# filter
biadjacency_cluster.data = biadjacency_cluster.data * (biadjacency_cluster.data >= 100)

In [ ]:
biadjacency_cluster.eliminate_zeros()

In [ ]:
image = svg_bigraph(biadjacency_cluster, names_row=names_labels_true, names_col=names[top_pages], 
                    display_edge_weight=True, display_node_weight=True)

In [ ]:
SVG(image)

## 3. Bipartite graphs

## Cinema

In [ ]:
graph = cinema

In [ ]:
biadjacency = graph.biadjacency
movies = graph.names_row
actors = graph.names_col

## To do

* Cluster the graph by Louvain (resolution 1). 
* List the 10 largest clusters and display the names of the top-5 actors and top-5 movies of these cluster in terms of Personalized PageRank.
* Give the strongest cluster among all clusters with at least 100 movies.

In [ ]:
louvain.fit(biadjacency)

In [ ]:
labels_movie = louvain.labels_row_
labels_actor = louvain.labels_col_

In [ ]:
for label in np.unique(labels_movie)[:10]:
    pagerank.fit(biadjacency, seeds_row=labels_movie==label, seeds_col=labels_actor==label)
    scores_movie = pagerank.scores_row_ * (labels_movie==label)
    scores_actor = pagerank.scores_col_ * (labels_actor==label)
    print(movies[top_k(scores_movie, 5)])
    print(movies[top_k(scores_actor, 5)])